#### Introduction to Hypothesis Testing

In [ ]:
# Calculate the proportion of late shipments
late_prop_samp = (late_shipments['late'] == "Yes").mean()

# Hypothesize that the proportion is 6%
late_prop_hyp = 0.06

# Calculate the standard error
std_error = np.std(late_shipments_boot_distn, ddof=1)

# Find z-score of late_prop_samp
z_score = (late_prop_samp - late_prop_hyp) / std_error

# Print z_score
print(z_score)

In [ ]:
# Calculate the p-value
p_value = 1 - norm.cdf(z_score)
                 
# Print the p-value
print(p_value) 

In [ ]:
# Calculate 95% confidence interval using quantile method
lower = np.quantile(late_shipments_boot_distn, 0.025)
upper = np.quantile(late_shipments_boot_distn, 0.975)

# Print the confidence interval
print((lower, upper))

In [ ]:
# Calculate the degrees of freedom
degrees_of_freedom = n_no + n_yes - 2

# Calculate the p-value from the test stat
p_value = t.cdf(t_stat, df=degrees_of_freedom)

# Print the p_value
print(p_value)

Paired t-tests \
<b>pingouin.ttest(x=sample_data["diff"], y=0, alternative="less")</b> two-sided, less, greater corresponding to two-tailed, left-tailed, and right-tailed. Y equals the hypothesized difference value from the null hypothesis \
<b>pingouin.ttest(x=sample_data["col1"], sample_data["col2"], paired=True, alternative="less")</b> Same as above \


ANOVA \
<b>pingouin.anova(data=data, dv="col_1", between="col_3")</b> DV is dependent variable and column of groups to calculate between \
<b>pingouin.pairwise_tests(data=data, dv="col_1", between="col_3", padjust="bonf")</b>

Two sample proportion tests \
Count the late column values for each freight_cost_group \
late_by_freight_cost_group = late_shipments.groupby("freight_cost_group")['late'].value_counts() \

Create an array of the "Yes" counts for each freight_cost_group \
success_counts = np.array([45, 16]) \

Create an array of the total number of rows in each freight_cost_group \
n = np.array([45 + 500, 16 + 439]) \

Run a z-test on the two proportions \
stat, p_value = proportions_ztest(count=success_counts, nobs=n, alternative="larger") \

The chi-square independence test compares proportions of successes of one categorical variable across the categories of another categorical variable.

In [ ]:
# Proportion of freight_cost_group grouped by vendor_inco_term
props = late_shipments.groupby('vendor_inco_term')['freight_cost_group'].value_counts(normalize=True)

# Convert props to wide format
wide_props = props.unstack()

# Proportional stacked bar plot of freight_cost_group vs. vendor_inco_term
wide_props.plot(kind="bar", stacked=True)
plt.show()

# Determine if freight_cost_group and vendor_inco_term are independent
expected, observed, stats = pingouin.chi2_independence(data=late_shipments, x="vendor_inco_term", y="freight_cost_group")

# Print results
print(stats[stats['test'] == 'pearson']) 

The chi-square goodness of fit test compares proportions of each level of a categorical variable to hypothesized values. Before running such a test, it can be helpful to visually compare the distribution in the sample to the hypothesized distribution.

In [ ]:
# Find the number of rows in late_shipments
n_total = len(late_shipments)

# Create n column that is prop column * n_total
hypothesized["n"] = hypothesized["prop"] * n_total

# Plot a red bar graph of n vs. vendor_inco_term for incoterm_counts
plt.bar(incoterm_counts['vendor_inco_term'], incoterm_counts['n'], color="red", label="Observed")

# Add a blue bar plot for the hypothesized counts
plt.bar(hypothesized['vendor_inco_term'], hypothesized['n'], alpha=0.5, color="blue", label="Hypothesized")
plt.legend()
plt.show()

Non parametric tests is used when sample size is not big enough. \
<b>wilcoxon_test_results = pingouin.wilcoxon(x=sample_dem_data['dem_percent_12'],  y=sample_dem_data['dem_percent_16'], alternative="two-sided")</b> Used instead of paired t-tests \
<b>use pingouin.mwu on unpaired t-tests</b> Shown below \ 
<b>use pingouin.kruskal on ANOVA test</b> Shown 2 cells below \


In [ ]:
# Select the weight_kilograms and late columns
weight_vs_late = late_shipments[["weight_kilograms", "late"]]

# Convert weight_vs_late into wide format
weight_vs_late_wide = weight_vs_late.pivot(columns='late', 
                                           values='weight_kilograms')

# Run a two-sided Wilcoxon-Mann-Whitney test on weight_kilograms vs. late
wmw_test = pingouin.mwu(x=weight_vs_late_wide['No'],
                        y=weight_vs_late_wide['Yes'],
                        alternative='two-sided')

In [ ]:
# Run a Kruskal-Wallis test on weight_kilograms vs. shipment_mode
kw_test = pingouin.kruskal(data=late_shipments, 
                           dv='weight_kilograms',
                           between='shipment_mode')